In [1]:
import pickle
import numpy as np
import pandas as pd
from datetime import datetime

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import keras_tuner as kt
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
with open(r'.\Reviews_Dataset_Splits\X_train.pkl', 'rb') as f:
    X_train = pickle.load(f)

with open(r'.\Reviews_Dataset_Splits\X_val.pkl', 'rb') as f:
    X_val = pickle.load(f)

with open(r'.\Reviews_Dataset_Splits\X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)

with open(r'.\Reviews_Dataset_Splits\y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)

with open(r'.\Reviews_Dataset_Splits\y_val.pkl', 'rb') as f:
    y_val = pickle.load(f)

with open(r'.\Reviews_Dataset_Splits\y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)

y_train = keras.utils.to_categorical(y_train-1, num_classes=5)
y_val = keras.utils.to_categorical(y_val-1, num_classes=5)
y_test = keras.utils.to_categorical(y_test-1, num_classes=5)

with open(r'.\embeddingMatrix_Reviews.pkl', 'rb') as f:
    embedding_matrix = pickle.load(f)

num_tokens = len(embedding_matrix) # total vocabulary +1 or length of embedding matrix
embedding_dim = 300 # dimension of the vector of a single word
MAX_REVIEW_LEN = 250 # maximum words in a review
num_classes = 5

In [14]:
def build_model(hp, max_layers, test_optimizers, test_activations, use_Dropout,
                filters_min_value, filters_max_value, filters_step):
    embedding_layer = keras.layers.Embedding(
        num_tokens,
        embedding_dim,
        embeddings_initializer=keras.initializers.Constant(embedding_matrix),
        input_length=MAX_REVIEW_LEN,
        trainable=True)
   
    model = keras.Sequential()
    model.add(embedding_layer)
    
    num_layers = hp.Int('num_layers', 3, max_layers)

    if test_activations:
            activation = hp.Choice(f'activation', ['softplus', 'softsign', 'relu', 'tanh'])
    else:
        activation = 'relu'  # Default activation

    for i in range(num_layers):
        filters = hp.Int(f'filters_{i}', min_value=filters_min_value, max_value=filters_max_value, step=filters_step)

        model.add(layers.Conv1D(
            filters=filters,
            kernel_size=5,
            activation=activation,
            padding='same'
        ))

        if use_Dropout:
            model.add(layers.Dropout(rate=hp.Choice(f'dropout_rate_{i}', [0.0, 0.2])))

        # Add pooling layer
        model.add(layers.MaxPooling1D(pool_size=2))

    model.add(layers.GlobalAveragePooling1D())
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    # Optimizer
    if test_optimizers:
        optimizer = hp.Choice('optimizer', ['SGD', 'RMSprop', 'Adam', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam'])
    else:
        optimizer = 'adam'
    
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [15]:
NUM_EPOCHS = 12
BATCH_SIZE = 128

now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
directory = f'KerasTuner_Logs/CNN/CNN_V1_3Layer_Density_Optimization_{now}'

# Callbacks
tensorboard = TensorBoard(log_dir=f'TensorBoard_Logs/CNN/CNN_V1_3Layer_Density_Optimization_{now}')
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)

tuner = kt.GridSearch(
    lambda hp: build_model(hp, max_layers=3, test_optimizers=False, test_activations=False, use_Dropout=True, 
                           filters_min_value=64, filters_max_value=128, filters_step=32),
    objective=kt.Objective("val_loss", direction="min"),
    max_trials=None,
    executions_per_trial=1,
    directory=directory,
    project_name='Reviews_Classification')

In [16]:
tuner.search(x=X_train,
             y=y_train,
             verbose=1,
             epochs=NUM_EPOCHS,
             batch_size=BATCH_SIZE,
             callbacks=[tensorboard, early_stopping],
             validation_data=(X_val, y_val))

Trial 177 Complete [00h 00m 48s]
val_loss: 1.0720566511154175

Best val_loss So Far: 1.0652364492416382
Total elapsed time: 02h 53m 31s

Search: Running Trial #178

Value             |Best Value So Far |Hyperparameter
3                 |3                 |num_layers
128               |96                |filters_0
0                 |0                 |dropout_rate_0
128               |96                |filters_1
0.2               |0.2               |dropout_rate_1
96                |64                |filters_2
0.2               |0.2               |dropout_rate_2

Epoch 1/12
151/771 [====>.........................] - ETA: 6s - loss: 1.4385 - accuracy: 0.3461

KeyboardInterrupt: 

In [17]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3,
 'filters_0': 96,
 'dropout_rate_0': 0.0,
 'filters_1': 96,
 'dropout_rate_1': 0.2,
 'filters_2': 64,
 'dropout_rate_2': 0.2}

In [7]:
tuner.get_best_models()[0].summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 250, 300)          16058400  
                                                                 
 global_average_pooling1d (G  (None, 300)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 128)               38528     
                                                                 
 dense_1 (Dense)             (None, 5)                 645       
                                                                 
Total params: 16,097,573
Trainable params: 16,097,573
Non-trainable params: 0
_________________________________________________________________


In [8]:
tuner.results_summary()

Results summary
Results in dir_2024-07-05_11-38-27\Reviews_Classification
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 0001 summary
Hyperparameters:
units: 85
num_layers: 1
optimizer: Adam
Score: 1.103798508644104

Trial 0003 summary
Hyperparameters:
units: 85
num_layers: 2
optimizer: Adam
Score: 1.1182894706726074

Trial 0000 summary
Hyperparameters:
units: 85
num_layers: 1
optimizer: RMSprop
Score: 1.1338261365890503

Trial 0002 summary
Hyperparameters:
units: 85
num_layers: 2
optimizer: RMSprop
Score: 1.1509822607040405
